In [1]:
# pip install cbpro 
import cbpro
import datetime
import pandas as pd
import psycopg2
import sqlalchemy
import getpass

In [2]:
# # basic psycopg2 usage (not directly compatible with postgresql) 
# conn = psycopg2.connect("dbname=finanz user=kenny")
# # Open a cursor to perform database operations
# cur = conn.cursor()
# # cur.execute("INSERT INTO symbols (string) VALUES('ETH-EUR');")
# cur.execute("SELECT * FROM symbols;")
# cur.fetchone()
# conn.commit() # persist the changes and clsoe the cursor and the connection 
# cur.close() 
# conn.close() 

In [3]:
# get password for kenny user 
def connect2DB(user):
    pwd = getpass.getpass("Get password for {}".format(user))
    # and create a connection to posgresql or use sqlalchemy 
    db_engine = sqlalchemy.create_engine('postgresql+psycopg2://{}:{}@localhost/finanz'.format(user,pwd), pool_recycle=3600)
    db_conn = db_engine.connect()
    return db_conn

In [4]:
TICKERS = ['ETH-EUR']

In [4]:
db_conn = connect2DB("kenny")

# Read data from PostgreSQL database table and load into a DataFrame instance
symbols = pd.read_sql("select * from symbols where symbol IN ({}) ".format(",".join(["'{}'".format(t) for t in TICKERS])), db_conn, )
candles = pd.read_sql("select * from candles where fk_symbol in ({})".format(",".join([str(i) for i in symbols.id])), db_conn)
columns = ['time','high','low','open','close','volume','resolution_sec', 'symbol','fk_symbol']
df = candles.merge(symbols, left_on = "fk_symbol", right_on = "id")[columns]
df.set_index(["symbol","time"], inplace = True)
db_conn.close()

Get password for kenny ············


In [5]:
symbols

,id,symbol
0,1,ETH-EUR


In [6]:
candles

,id,time,high,low,open,close,volume,fk_symbol,resolution_sec


In [7]:
# api for getting historical data from cbpro
pclient = cbpro.PublicClient()

now_there = datetime.datetime.fromisoformat(pclient.get_time()["iso"][0:-1]) # remove last item
now_here = datetime.datetime.fromisoformat(datetime.datetime.now().isoformat())

resolution = {"1min": 60 , "5min" : 300, "15min" : 900, "1hr" : 3600, "6hr" : 21600, "1day" : 86400}
MAX_TICKERS = 200


def reshape_data_for_analysis(df):
    return df.merge(symbols, left_on='symbol', right_on = "symbol").rename(columns = {"id":"fk_symbol"}).set_index(["symbol","time"])
    
def reshape_data_for_db(df):
    return df.reset_index().drop(['symbol'], axis = 1)

def convert_candles_to_df(candles, ticker, granularity):
    """
    convert the data candles from the raw coinbase request to dataframe 
    
    `index = time , columns = { low, high, open, close, volume },
     [ 1415398768, 0.32, 4.2, 0.35, 4.2, 12.3 ],
    :param: candles 
    :return: dataframe 
    """
    df= pd.DataFrame(columns = ["time","low","high","open","close","volume"], data = candles)
    df["time"] = df["time"].apply(lambda x : datetime.datetime.fromtimestamp(x))
    df["symbol"] = ticker
    df["resolution_sec"] = resolution[granularity]
    return reshape_data_for_analysis(df)
  

def get_num_candles(begin, end, granularity):
    """
    how many candles of `granularity` are in the interval [begin; end]
    """
    return int((end - begin).total_seconds() // resolution[granularity])

def get_data(client, ticker, begin, end,granularity):
    """
    fetch historical ticker data for the time period [begin;end] with granularity `granularity`
    :param: client the api client instance to query
    :param: ticker the ticker to fetch data for 
    :param: begin the time from which to get data 
    :param: end the time to which to get data 
    :param: granularity a string representing the granularity of the data, i.e. `1min`, `5min` etc.
    :return: directly the cbpro api response (a list of lists of values [ time, low, high, open, close, volume ])
    """
    return client.get_product_historic_rates(ticker, 
                                             start = begin.isoformat(), 
                                             end = end.isoformat(),
                                             granularity = resolution[granularity])

def get_data_from(client, ticker, begin, size, granularity):
    """
     fetch the `size` candles of `granularity` resolution, starting from `begin` timestamp
     
     :param: client the api client instance to query
     :param: ticker the ticker to fetch data for 
     :param: begin the time from which to get data 
     :param: size the number of candles to fetch 
     :param: granularity a string representing the granularity of the data, i.e. `1min`, `5min` etc.
     :return: a dataframe with the following columns (see example below)
     
     [ time, low, high, open, close, volume ],
     [ 1415398768, 0.32, 4.2, 0.35, 4.2, 12.3 ],
    """
    # size = q*MAX_TICKERS + r 
    q = size // MAX_TICKERS
    r = size % MAX_TICKERS 
    
    candles = []
    # get q * MAX_TICKERS data
    for i in range(q):
        end = (begin + datetime.timedelta(seconds = MAX_TICKERS * resolution[granularity]))
        print("Getting data from {} to {}".format(begin, end))
        candles = candles + get_data(client,ticker, begin, end, granularity)
        begin = end
      
    # get residual data
    end = (begin + datetime.timedelta(seconds = r * resolution[granularity]))
    candles = candles + get_data(client, ticker, begin, end, granularity)    
    return convert_candles_to_df(candles, ticker, granularity)
    
    
def get_data_to(client, ticker, end, size, granularity):
    """
     fetch the last `size` candles of `granularity` resolution, with respect to `end` timestamp
     
     :param: client the api client instance to query
     :param: ticker the ticker to fetch data for 
     :param: end the time to which to get data 
     :param: size the number of candles to fetch 
     :param: granularity a string representing the granularity of the data, i.e. `1min`, `5min` etc.
     :return: a dataframe with the following columns (see example below)
     
     [ time, low, high, open, close, volume ],
     [ 1415398768, 0.32, 4.2, 0.35, 4.2, 12.3 ],
    """
    # size = q*MAX_TICKERS + r 
    q = size // MAX_TICKERS
    r = size % MAX_TICKERS 
       
    candles = []
    # get q * MAX_TICKERS data
    for i in range(q):
        begin = (end - datetime.timedelta(seconds = MAX_TICKERS * resolution[granularity]))
        print("Getting data from {} to {}".format(begin, end))
        candles = candles + get_data(client,ticker, begin, end, granularity)
        end = begin
    
    # get residual data
    begin = (end - datetime.timedelta(seconds = r * resolution[granularity]))
    candles = candles + get_data(client, ticker, begin, end, granularity)    
    return convert_candles_to_df(candles, ticker, granularity)

def get_data_range(client, ticker, begin, end, granularity):
    size = get_num_candles(begin, end, granularity)
    return get_data_from(client, ticker, begin, size, granularity)



In [8]:
# fetch 15 minute resolution data from 1st of Jan - 7 am to 12th of Feb 19:00 to now 
TICKER = "ETH-EUR"
granularity = "5min"

begin = datetime.datetime(2020,11,1)
end = datetime.datetime(2021,2, 15)


In [9]:
get_num_candles(begin, end, granularity)
# df2.drop_duplicates()

30528

In [10]:
df2 = get_data_range(pclient, TICKER, begin, end, granularity).drop_duplicates()

Getting data from 2020-11-01 00:00:00 to 2020-11-01 16:40:00
Getting data from 2020-11-01 16:40:00 to 2020-11-02 09:20:00
Getting data from 2020-11-02 09:20:00 to 2020-11-03 02:00:00
Getting data from 2020-11-03 02:00:00 to 2020-11-03 18:40:00
Getting data from 2020-11-03 18:40:00 to 2020-11-04 11:20:00
Getting data from 2020-11-04 11:20:00 to 2020-11-05 04:00:00
Getting data from 2020-11-05 04:00:00 to 2020-11-05 20:40:00
Getting data from 2020-11-05 20:40:00 to 2020-11-06 13:20:00
Getting data from 2020-11-06 13:20:00 to 2020-11-07 06:00:00
Getting data from 2020-11-07 06:00:00 to 2020-11-07 22:40:00
Getting data from 2020-11-07 22:40:00 to 2020-11-08 15:20:00
Getting data from 2020-11-08 15:20:00 to 2020-11-09 08:00:00
Getting data from 2020-11-09 08:00:00 to 2020-11-10 00:40:00
Getting data from 2020-11-10 00:40:00 to 2020-11-10 17:20:00
Getting data from 2020-11-10 17:20:00 to 2020-11-11 10:00:00
Getting data from 2020-11-11 10:00:00 to 2020-11-12 02:40:00
Getting data from 2020-1

In [11]:
df2

low     high     open    close      volume  \
symbol  time                                                                  
ETH-EUR 2020-11-01 17:40:00   334.03   334.63   334.11   334.18   26.037805   
        2020-11-01 17:35:00   334.03   334.38   334.29   334.23   10.452691   
        2020-11-01 17:30:00   333.92   334.27   333.93   334.26    9.129421   
        2020-11-01 17:25:00   333.67   334.82   334.66   334.00   32.089240   
        2020-11-01 17:20:00   334.68   335.00   334.99   334.68   11.506148   
...                              ...      ...      ...      ...         ...   
        2021-02-14 14:45:00  1501.00  1510.31  1506.86  1501.79  155.390147   
        2021-02-14 14:40:00  1502.44  1511.30  1511.30  1507.45  226.593499   
        2021-02-14 14:35:00  1509.67  1519.41  1519.26  1511.53  138.201049   
        2021-02-14 14:30:00  1516.79  1520.10  1520.10  1518.44   41.633719   
        2021-02-14 14:25:00  1518.92  1521.74  1521.68  1520.10   73.691433   

                             resolution_sec  fk_symbol  
symbol  time                                            
ETH-EUR 2020-11-01 17:40:00             300          1  
        2020-11-01 17:35:00             300          1  
        2020-11-01 17:30:00             300          1  
        2020-11-01 17:25:00             300          1  
        2020-11-01 17:20:00             300          1  
...                                     ...        ...  
        2021-02-14 14:45:00             300          1  
        2021-02-14 14:40:00             300          1  
        2021-02-14 14:35:00             300          1  
        2021-02-14 14:30:00             300          1  
        2021-02-14 14:25:00             300          1  

[30501 rows x 7 columns]

In [12]:
# again connect to the DB
db_conn = connect2DB("kenny")

Get password for kenny ············


In [13]:
# this is how you store the data for sql
reshape_data_for_db(df2).to_sql("candles", db_conn, if_exists = "append", index = False) 
db_conn.close()
# ?df3.to_sql